In [2]:
import torch

# DICE evaluation metric
In the lab semantic segmentation, you have implemented IOU to evaluate the performance of the model. Here, you need to implement a similar evaluation metric called DICE or Sørensen–Dice coefficient, and it is formulated as: $$ DICE(X, X_{truth}) = \frac{2|X \cap X_{truth}|}{|X| + |X_{truth}|}$$ 

In [ ]:
def dice_coeff(inp : Tensor, tgt : Tensor):
    """
    inp: predicted mask
    tgt: Ground truth mask
    """
    eps = 1e-5 # small number to add to denominator to avoid division by zero
    #YOUR CODE START HERE
    sum_dim = (-1, -2, -3)
    # calculation of intersection   
    inter = 2 *(inp * tgt).sum(dim=sum_dim)

    # calculate the sum of |inp| + |tgt|
    sets_sum = inp.sum(dim=sum_dim) + tgt.sum(dim=sum_dim)
    sets_sum = torch.where(sets_sum == 0, inter, sets_sum)

    # calcaute the dice    
    dice = (inter + eps) / (sets_sum + eps)
    
    # average the dice of classwise
    return dice.mean()

Based on their formulation, what are the **difference** among Cross-entropy, DICE, and IOU?  

Your answer starts here

# Evaluate the model

Now report the Cross-entropy, DICE, and IOU for a model of your preference. You can use a pretrained model from the lab or train new model, but keep in mind that your model is able to take in inputs of size $(B, 3, 128, 128)$ and output prediction of $(B, C, 128, 128)$, where $B$ is batch size and $C$ is number of classes (8 in our case). The preprocessed image and masks from the lab are loaded as Torch.tensor below:

In [5]:
input_images = torch.load('input_images.pth')
masks = torch.load('masks.pth')

In [ ]:
#YOUR CODE START HERE